This Notebook evaluates the study comparing the different variants of numerical fluxes for a Mach 2 supersonic flow over a wedge as presented in the phd thesis "Implicit Discontinuous Galerkin Shock Tracking methods for Compressible Flows with Shocks" (Vandergrift 2024) 

In [1]:
#r ".\binaries\BoSSSpad.dll"
#r ".\binaries\XESF.dll"
using System;
using BoSSS.Foundation;
using BoSSS.Application.BoSSSpad;
using BoSSS.Solution;
using BoSSS.Solution.Tecplot;
using BoSSS.Foundation.IO;
using ilPSP.Tracing;
using BoSSS.Solution.Utils;
using ilPSP.LinSolvers;
using BoSSS.Solution.NSECommon;
using ilPSP.Connectors.Matlab;
using ilPSP;
using BoSSS.Foundation.Grid.Classic;
using ilPSP.Utils;
using BoSSS.Foundation.Grid.RefElements;
using System.Collections.Generic;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.XDG;
using BoSSS.Solution.XdgTimestepping;
using BoSSS.Solution.AdvancedSolvers;
using System.Linq;
using BoSSS.Foundation.Grid.Aggregation;
using BoSSS.Platform;
using MPI.Wrappers;
using System.Diagnostics;
using System.IO;
using System.Collections;
using BoSSS.Foundation.Quadrature;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Gnuplot;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using ApplicationWithIDT;
using XESF;
Init();


init workflow


In [2]:
BoSSSshell.WorkflowMgm.Init("XESF_WedgeFlow_FluxStudy");
BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();
BoSSSshell.WorkflowMgm.DefaultDatabase


Project name is set to 'XESF_WedgeFlow_FluxStudy'.
Default Execution queue is chosen for the database.
Opening existing database '\\dc3\userspace\sebastian\cluster\XESF_WedgeFlow_FluxStudy'.


{ Session Count = 4; Grid Count = 4; Path = \\dc3\userspace\sebastian\cluster\XESF_WedgeFlow_FluxStudy }

look at all sessions

In [3]:

var dbPath = BoSSSshell.WorkflowMgm.DefaultDatabase.Path;
var database = OpenOrCreateDatabase(dbPath);
var sessions = database.Sessions;
sessions


#0: XESFWedgeFlow_TwoLs	XDGWS-15x10-agg0.4-iPrb0-iFlx3-wFLx0-bFlx0-Fphi0-wA10	4/3/2024 7:21:02 PM	2428d567...
#1: XESFWedgeFlow_TwoLs	XDGWS-15x10-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0-wA10	4/3/2024 7:20:27 PM	77b424e6...
#2: XESFWedgeFlow_TwoLs	XDGWS-15x10-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0-wA10	4/3/2024 7:17:29 PM	28b98cfa...
#3: XESFWedgeFlow_TwoLs	XDGWS-15x10-agg0.4-iPrb0-iFlx2-wFLx0-bFlx0-Fphi0-wA10	4/3/2024 7:20:47 PM	934d3c2c...


pick one session and timestep to compute the exact enthalpy

In [4]:
MatrixAssembler aaa = new MatrixAssembler(3);
var si = database.Sessions.Pick(0);
var ti = si.Timesteps;

## Enthalpy error
enthalpy is computed as
$$h=\frac{(\rho E + p)}{\rho},~ p=(\gamma -1)(\rho E - \rho e_{kin}),~\rho e_{kin}=\frac{\rho }{2}(\vert u\vert^2 + \vert v\vert^2)$$

For an uniform inviscid flow the inflow enthalpy should remain constant throughout the flowfield 

In the case of the WedgeFlow ($p_\text{in}=1,\rho_\text{in}=1$) presented, the v-velocity is zero and we have
$$ h_{in} = \rho E_{in}+1$$
We prescribe the inflow conditions by the Mach number $M_{in}=2$. From those the Energy is computed as 
$$ \rho E_{in} = (\rho e_{inner})_\text{in}+(\rho e_{kin})_\text{in}=\frac{p_\text{in}}{\gamma-1}+\frac{\rho_\text{in}}{2}u_\text{in}^2= \frac{1}{\gamma-1}+\frac{1}{2}u_{in}^2$$ 
where $u_{in} = M_{in}\sqrt{\gamma}$. In total we obtain:
$$ h_\text{in}=\rho E_{in}+1=\frac{\gamma}{\gamma-1}+\frac{1}{2}u_{in}^2$$


In [5]:
using BoSSS.Solution.CompressibleFlowCommon.MaterialProperty;
var enthalpy_end = (XDGField) si.Timesteps.Last().GetField("h");
var enthalpy_exact = (XDGField) enthalpy_end.Clone();
double gamma = IdealGas.Air.HeatCapacityRatio;
double Ms = 2.0;
double densityLeft = 1;
double pressureLeft = 1;
double velocityXLeft = Ms * Math.Sqrt(gamma * pressureLeft / densityLeft);

(gamma)/(gamma-1) + 0.5*velocityXLeft*velocityXLeft

6.300000000000001

In [6]:
var enthalpy_inflow = (gamma)/(gamma-1) + 0.5*velocityXLeft*velocityXLeft;
enthalpy_exact.Clear();
enthalpy_exact.GetSpeciesShadowField("L").ProjectField(1.0,t => enthalpy_inflow);
enthalpy_exact.GetSpeciesShadowField("R").ProjectField(1.0,t => enthalpy_inflow);
enthalpy_exact.Identification= "h_exact";
var EEN = enthalpy_exact.L2NormAllSpecies();

## Choose which runs to plot

In [7]:
//var sessions = database.Sessions.Where(s => s.Timesteps.Count >0); //plot all 
var ses=sessions.Where(s => (s.SuccessfulTermination &&s.Name.Contains("iPrb0"))); // the one plotted in the publication
ses


#0: XESFWedgeFlow_TwoLs	XDGWS-15x10-agg0.4-iPrb0-iFlx3-wFLx0-bFlx0-Fphi0-wA10	4/3/2024 7:21:02 PM	2428d567...
#1: XESFWedgeFlow_TwoLs	XDGWS-15x10-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0-wA10	4/3/2024 7:20:27 PM	77b424e6...
#2: XESFWedgeFlow_TwoLs	XDGWS-15x10-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0-wA10	4/3/2024 7:17:29 PM	28b98cfa...
#3: XESFWedgeFlow_TwoLs	XDGWS-15x10-agg0.4-iPrb0-iFlx2-wFLx0-bFlx0-Fphi0-wA10	4/3/2024 7:20:47 PM	934d3c2c...


# A Helper Function to obtain different plot formats

In [8]:
using BoSSS.Solution.Tecplot;
using System.IO;
var texplot = new Tecplot(enthalpy_exact.GridDat, 2);
var allEE = new List<List<double>>();
var plot = new Plot2Ddata();
public PlotFormat GetFormat(int count){
    
    var allPT = new BoSSS.Solution.Gnuplot.PointTypes[] { PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle,PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle,PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle,PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle,PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle,PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle};
    var allC = new BoSSS.Solution.Gnuplot.LineColors[] { LineColors.Blue, LineColors.Black, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red};
    var Fmt = new PlotFormat();
    Fmt.PointSize = 0.8;
    Fmt.LineWidth = 1;    
    Fmt.Style     = Styles.LinesPoints;

    Fmt.LineColor = allC[count];
    Fmt.PointType =  allPT[count];
    return Fmt;
}

## Actual Etnhalpy Error Plot


In [9]:
int count=0;
foreach(var si in ses){
    var enthalpyerrors = new List<double>();
    foreach(var timestep in si.Timesteps){
        var error = ((XDGField) timestep.GetField("h_err")).L2NormAllSpecies()/EEN;
        enthalpyerrors.Add(error);
    }
    allEE.Add(enthalpyerrors);
    IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) si.Timesteps.Pick(enthalpyerrors.Count -1)).GetInternal();
    plot.AddDataGroup(si.Name,lts.TimeStepNumbers,enthalpyerrors, GetFormat(count));
    count++;
    enthalpyerrors.SaveToTextFile($"EnthalpyErrors_{si.Name}.txt");
}
plot.Xlabel="Iteration";
plot.LogY = true;
plot.ShowXtics = true;
plot.ToGnuplot().PlotSVG(xRes:1200,yRes:600)

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Note: In a Jupyter Worksheet, you must NOT have a trailing semicolon in order to see the plot on screen; otherwise, the output migth be surpressed.!


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -16 
 
 
 
 
 10 -14 
 
 
 
 
 10 -12 
 
 
 
 
 10 -10 
 
 
 
 
 10 -8 
 
 
 
 
 10 -6 
 
 
 
 
 10 -4 
 
 
 
 
 10 -2 
 
 
 
 
 10 0 
 
 
 
 
 0 
 
 
 
 
 5 
 
 
 
 
 10 
 
 
 
 
 15 
 
 
 
 
 20 
 
 
 
 
 25 
 
 
 
 
 30 
 
 
 
 
 35 
 
 
 
 
 40 
 
 
 
 
 45 
 
 
 
 
 50 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Iteration 
 
 
 
 
 XDGWS-15x10-agg0.4-iPrb0-iFlx3-wFLx0-bFlx0-Fphi0-wA10 
 
 
 XDGWS-15x10-agg0.4-iPrb0-iFlx3-wFLx0-bFlx0-Fphi0-wA10 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M489.0,57.1 L542.4,57.1 M53.9,50.9 L75.8,52.8 L97.7,53.2 L119.7,54.1 L141.6,63.6 L163.5,67.6
 L185.4,77.1 L207.4,78.0 L229.3,87.6 L251.2,89.4 L273.1,93.4 L295.1,97.3 L317.0,128.0 L338.9,137.9
 L360.8,146.6 L382.8,163.0 L404.7,225.9 L426.6,197.7 L448.5,201.5 L470.5,221.1 L492.4,220.9 L514.3,219.5
 L536.2,203.6 L558.2,204.1 L580.1,204.2 L602.0,204.5 L623.9,205.1 L645.8,206.5 L667.8,206.6 L689.7,206.6
 L711.6,204.9 L733.5,205.6 L755.5,205.7 L777.4,206.1 L799.3,206.4 L821.2,206.6 L843.2,206.8 L865.1,206.8
 L887.0,206.8 L908.9,206.8 L930.9,206.8 L952.8,206.8 L974.7,206.8 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 XDGWS-15x10-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0-wA10 
 
 
 XDGWS-15x10-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0-wA10 
 
 
 
	<path stroke='rgb( 0, 0, 0)' d='M489.0,81.1 L542.4,81.1 M53.9,50.9 L75.8,52.8 L97.7,53.2 L119.7,54.1 L141.6,63.6 L163.5,67.6
 L185.4,77.1 L207.4,78.0 L229.3,87.6 L251.2,91.5 L273.1,124.4 L295.1,128.2 L317.0,135.2 L338.9,141.5
 L360.8,152.8 L382.8,144.9 L404.7,154.5 L426.6,163.5 L448.5,201.1 L470.5,272.6 L492.4,402.5 L514.3,451.2
 L536.2,500.4 L558.2,514.7 L580.1,525.3 L602.0,530.5 L623.9,530.6 L645.8,531.8 L667.8,531.7 L689.7,531.4
 L711.6,531.4 L733.5,531.4 L755.5,531.4 L777.4,531.4 L799.3,531.4 L821.2,531.4 L843.2,531.4 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 XDGWS-15x10-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0-wA10 
 
 
 XDGWS-15x10-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0-wA10 
 
 
 
	<path stroke='rgb(255, 0, 0)' d='M489.0,105.1 L542.4,105.1 M53.9,50.9 L75.8,52.8 L97.7,53.2 L119.7,54.1 L141.6,63.6 L163.5,67.6
 L185.4,77.1 L207.4,78.0 L229.3,87.6 L251.2,91.5 L273.1,123.9 L295.1,127.7 L317.0,129.4 L338.9,135.6
 L360.8,137.4 L382.8,143.4 L404.7,145.3 L426.6,155.2 L448.5,175.3 L470.5,247.6 L492.4,324.4 L514.3,349.3
 L536.2,363.3 L558.2,377.6 L580.1,383.1 L602.0,385.3 L623.9,386.1 L645.8,386.7 L667.8,386.9 L689.7,387.0
 L711.6,387.0 L733.5,387.1 L755.5,387.1 L777.4,387.1 L799.3,387.1 L821.2,387.1 L843.2,387.1 L865.1,387.1
 L887.0,387.1 L908.9,387.1 L930.9,387.1 L952.8,387.1 L974.7,387.1 L996.6,387.1 L1018.6,387.1 L1040.5,387.1
 L1062.4,387.1 L1084.3,387.1 L1106.3,387.1 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 XDGWS-15x10-agg0.4-iPrb0-iFlx2-wFLx0-bFlx0-Fphi0-wA10 
 
 
 XDGWS-15x10-agg0.4-iPrb0-iFlx2-wFLx0-bFlx0-Fphi0-wA10

## Residual History

In [10]:

using BoSSS.Solution.Tecplot;
using System.IO;
//Directory.GetFiles(".", "*.plt").ForEach(file => File.Delete(file));
var texplot = new Tecplot(enthalpy_exact.GridDat, 2);
var allEE = new List<List<double>>();
var plot = new Plot2Ddata();
int count=0;
foreach(var si in ses){
    var enthalpyerrors = new List<double>();
    IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) si.Timesteps.Last()).GetInternal();
    lts.ResHistory.SaveToTextFile($"ResHistory_{si.Name}.txt");
    lts.ResHistory.SaveToTextFile($"EnResHistory_{si.Name}.txt");
    plot.AddDataGroup("||r|| - "+ si.Name,lts.TimeStepNumbers,lts.ResHistory, GetFormat(count));
    count++;
    plot.AddDataGroup("EnRes_"+si.Name,lts.TimeStepNumbers,lts.EnResHistory, GetFormat(count));
    count++;
}

plot.Xlabel="Iteration";
plot.LogY = true;
plot.ShowXtics = true;
plot.ToGnuplot().PlotSVG(xRes:800,yRes:500)

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Note: In a Jupyter Worksheet, you must NOT have a trailing semicolon in order to see the plot on screen; otherwise, the output migth be surpressed.!


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -14 
 
 
 
 
 10 -12 
 
 
 
 
 10 -10 
 
 
 
 
 10 -8 
 
 
 
 
 10 -6 
 
 
 
 
 10 -4 
 
 
 
 
 10 -2 
 
 
 
 
 10 0 
 
 
 
 
 10 2 
 
 
 
 
 0 
 
 
 
 
 5 
 
 
 
 
 10 
 
 
 
 
 15 
 
 
 
 
 20 
 
 
 
 
 25 
 
 
 
 
 30 
 
 
 
 
 35 
 
 
 
 
 40 
 
 
 
 
 45 
 
 
 
 
 50 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Iteration 
 
 
 
 
 ||r|| - XDGWS-15x10-agg0.4-iPrb0-iFlx3-wFLx0-bFlx0-Fphi0-wA10 
 
 
 ||r|| - XDGWS-15x10-agg0.4-iPrb0-iFlx3-wFLx0-bFlx0-Fphi0-wA10 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M0.2,57.1 L53.6,57.1 M53.9,53.1 L67.8,54.5 L81.7,54.8 L95.7,55.5 L109.6,61.8 L123.5,66.3
 L137.4,72.9 L151.4,73.4 L165.3,81.9 L179.2,83.3 L193.1,83.4 L207.1,88.7 L221.0,90.8 L234.9,100.2
 L248.8,108.0 L262.8,123.3 L276.7,133.0 L290.6,137.3 L304.5,139.3 L318.5,140.1 L332.4,140.5 L346.3,141.1
 L360.2,154.6 L374.2,154.9 L388.1,155.0 L402.0,155.1 L415.9,155.3 L429.8,156.8 L443.8,156.9 L457.7,156.9
 L471.6,158.9 L485.5,159.1 L499.5,159.2 L513.4,159.4 L527.3,159.5 L541.2,165.1 L555.2,165.2 L569.1,165.3
 L583.0,165.3 L596.9,165.3 L610.9,165.3 L624.8,165.3 L638.7,165.3 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 EnResXDGWS-15x10-agg0.4-iPrb0-iFlx3-wFLx0-bFlx0-Fphi0-wA10 
 
 
 EnRes X DGWS-15x10-agg0.4-iPrb0-iFlx3-wFLx0-bFlx0-Fphi0-wA10 
 
 
 
	<path stroke='rgb( 0, 0, 0)' d='M0.2,81.1 L53.6,81.1 M53.9,45.7 L67.8,47.2 L81.7,47.6 L95.7,48.1 L109.6,53.8 L123.5,58.3
 L137.4,63.5 L151.4,64.1 L165.3,67.9 L179.2,69.0 L193.1,70.0 L207.1,72.9 L221.0,77.0 L234.9,83.8
 L248.8,91.7 L262.8,114.1 L276.7,120.9 L290.6,118.1 L304.5,120.1 L318.5,120.0 L332.4,120.1 L346.3,120.2
 L360.2,118.6 L374.2,118.8 L388.1,118.8 L402.0,118.9 L415.9,119.0 L429.8,119.4 L443.8,119.4 L457.7,119.4
 L471.6,118.9 L485.5,119.1 L499.5,119.1 L513.4,119.2 L527.3,119.2 L541.2,119.1 L555.2,119.2 L569.1,119.2
 L583.0,119.2 L596.9,119.2 L610.9,119.2 L624.8,119.2 L638.7,119.2 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ||r|| - XDGWS-15x10-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0-wA10 
 
 
 ||r|| - XDGWS-15x10-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0-wA10 
 
 
 
	<path stroke='rgb(255, 0, 0)' d='M0.2,105.1 L53.6,105.1 M53.9,53.1 L67.8,54.5 L81.7,54.8 L95.7,55.4 L109.6,62.0 L123.5,66.3
 L137.4,72.7 L151.4,73.2 L165.3,81.9 L179.2,83.0 L193.1,87.9 L207.1,93.0 L221.0,90.7 L234.9,97.7
 L248.8,98.2 L262.8,117.4 L276.7,121.5 L290.6,120.4 L304.5,142.4 L318.5,201.6 L332.4,337.4 L346.3,335.7
 L360.2,348.0 L374.2,359.8 L388.1,371.8 L402.0,383.7 L415.9,395.9 L429.8,408.6 L443.8,420.5 L457.7,424.2
 L471.6,424.2 L485.5,424.2 L499.5,424.2 L513.4,424.2 L527.3,424.2 L541.2,424.2 L555.2,424.2 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 EnResXDGWS-15x10-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0-wA10 
 
 
 EnRes X DGWS-15x10-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0-wA10 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M0.2,129.1 L53.6,129.1 M53.9,45.7 L67.8,47.2 L81.7,47.6 L95.7,48.1 L109.6,54.0 L123.5,58.3
 L137.4,63.4 L151.4,64.0 L165.3,67.9 L179.2,69.2 L193.1,71.5 L207.1,75.0 L221.0,78.2 L234.9,81.5
 L248.8,86.1 L262.8,89.1 L276.7,95.8 L290.6,102.4 L304.5,130.8 L318.5,191.1 L332.4,293.3 L346.3,323.8
 L360.2,333.2 L374.2,345.1 L388.1,357.0 L402.0,368.9 L415.9,381.7 L429.8,393.3 L443.8,406.1 L457.7,407.8
 L471.6,407.8 L485.5,407.8 L499.5,407.8 L513.4,407.8 L527.3,407.8 L541.2,407.8 L555.2,407.8 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ||r|| - XDGWS-15x10-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0-wA10 
 

### Plot ShadowFields

In [11]:
BoSSS.Solution.Application.DeleteOldPlotFiles();
foreach(var si in ses){
    IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) si.Timesteps.Last()).GetInternal();
    var flds=lts.GetShadowFields();
    var tp= new Tecplot(flds.Last().GridDat,2);
    tp.PlotFields(si.Name,0,flds);
}

rm;
